# DM lesson4

Pour le 23 octobre 2019

Dans ce dataset: https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv, chaque ligne correspond à un produit alimentaire mis en vente par un utilisateur.

Objectif: cleaner le dataset.

On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.
La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.
Voic une liste d'APIs qui peut vous être utile : https://github.com/public-apis/public-apis (mais vous pouvez en utiliser d'autres si vous le voulez).

In [2]:
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../../fspot-infmdi-721/lesson5/products.csv', sep=';')

### Get the approximate currency from IP

In [4]:
def getCurrencyFromIp(ip):
    if re.match('(\d{2,3}\.){3}\d{2,3}', ip):
        response = requests.get('http://ip-api.com/json/%s?fields=status,currency,country,city' % ip)
        if response.status_code == 200:
            reverseGeoInfo = response.json()
            if reverseGeoInfo['status'] == 'success':
                return reverseGeoInfo['currency']
    return None

Test

In [5]:
getCurrencyFromIp('666.666.666.666'), getCurrencyFromIp('26.191.237.49'), getCurrencyFromIp('58.90.204.239'), getCurrencyFromIp('nope')

(None, 'USD', 'JPY', None)

Apply

In [6]:
df['currency'] = df['ip_address'].apply(getCurrencyFromIp)

In [7]:
df.head(10)

,username,ip_address,product,price,infos,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,None
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,Ingredients: sugar and milk,None
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,None
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MXN
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MXN
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",USD


In [8]:
df['currency'].isnull().describe()

count      200
unique       2
top       True
freq       165
Name: currency, dtype: object

### Get the Euro rates and compute prices in Euro

In [9]:
ratesToEuro = requests.get('https://api.exchangerate-api.com/v4/latest/EUR').json()['rates']

In [10]:
def convertToEuro(price, currency):
    if currency in ratesToEuro:
        return price / ratesToEuro[currency]
    return -1

In [11]:
convertToEuro(949.79, 'JPY')

7.8412746820506305

In [12]:
df['price'] = pd.to_numeric(df.price, errors='coerce')

In [13]:
df['price_euro'] = df[['price', 'currency']].apply(lambda args: convertToEuro(args[0], args[1]), axis=1)

In [14]:
df.head(20)

,username,ip_address,product,price,infos,currency,price_euro
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.80,May contain sugar,None,-1.000000
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None,-1.000000
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None,-1.000000
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,313.854036
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY,7.841275
5,bsnozzwell5,226.52.32.70,Oil - Sesame,NaN,Ingredients: sugar and milk,None,-1.000000
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,None,-1.000000
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MXN,5.225554
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MXN,4.124328
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",USD,771.974264


### Extraction de la liste d'ingrédients

#### Cleanup and split the lists of ingredients

In [29]:
df['ingredients'] = df['infos'].str.replace('Ingredients:|Contains|May contain', ''). \
    str.replace(' and ', ',').str.replace(' ', '').str.split(',')

#### List all possible ingredients

In [33]:
allIngredients = set()
def countAllIngredients(ingredients):
    for i in ingredients:
        if len(i) > 0 and i not in allIngredients:
           allIngredients.add(i)

In [35]:
df['ingredients'].apply(countAllIngredients);

In [36]:
allIngredients

{'egg', 'fish', 'gluten', 'milk', 'mustard', 'peanut', 'soja', 'sugar'}

#### One hot encoding of the ingredients

In [39]:
for i in allIngredients:
    df[i] = df['ingredients'].apply(lambda x: i in x)

In [40]:
df.head(10)

,username,ip_address,product,price,infos,currency,price_euro,ingredients,egg,soja,milk,gluten,sugar,peanut,mustard,fish
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.80,May contain sugar,None,-1.000000,[sugar],False,False,False,False,True,False,False,False
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None,-1.000000,"[peanut, fish]",False,False,False,False,False,True,False,True
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None,-1.000000,"[mustard, fish]",False,False,False,False,False,False,True,True
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,313.854036,[gluten],False,False,False,True,False,False,False,False
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY,7.841275,"[sugar, egg, fish]",True,False,False,False,True,False,False,True
5,bsnozzwell5,226.52.32.70,Oil - Sesame,NaN,Ingredients: sugar and milk,None,-1.000000,"[sugar, milk]",False,False,True,False,True,False,False,False
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,None,-1.000000,[sugar],False,False,False,False,True,False,False,False
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MXN,5.225554,[sugar],False,False,False,False,True,False,False,False
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MXN,4.124328,"[sugar, milk, fish]",False,False,True,False,True,False,False,True
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",USD,771.974264,"[peanut, sugar, milk, fish]",False,False,True,False,True,True,False,True
